In [1]:
from submodulos.tokenizer.custom_tokenizer import SpacyCustomTokenizer
from submodulos.data import get_all_text_with_details, fb
from tools import get_progressbar
from datetime import datetime
import os
from word_db import WordDB

word_db = WordDB(localhost='localhost', port=6379, db=2)
corpus = fb.get_details_text(f'{os.getcwd()}/submodulos/data/')
name = 'facebook'


texts = get_all_text_with_details(f'{os.getcwd()}/submodulos/data/')

text_list = {
    'CubaDebate': 0,
    'Facebook': 0,
    'Twitter': 0,
    'Telegram': 0
}

for t, d, n in texts:
    text_list[n] += 1

text_list

/Users/dany/.local/share/virtualenvs/cuba-digital-lang-9Kd57kqM/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'CubaDebate': 63452, 'Facebook': 287, 'Twitter': 109941, 'Telegram': 156440}

In [55]:
class Data:
    date_distribution = {}
    date_size = {}
    
    def __init__(self, name):
        self.name = name

    def save(self):
        result = '{\n'
        result += f'"date_distribution": {str(self.date_distribution)},\n'
        result += f'"date_size": {str(self.date_size)},\n'
        result += '}'

        with open(f'results/{self.name}.json', 'w+') as f:
            f.write(result)
            f.close()


def print_best_word(list_):
    for i in range(20):
        print(list_[i], end=' ')





In [4]:
nlp = SpacyCustomTokenizer()

error: nothing to repeat at position 9631

In [ ]:

bar = get_progressbar(len(corpus), f' from {len(corpus)} text, tokenizer and tag with date ')
bar.start()
token_list = []
for i, data in enumerate(corpus):
    t, d, n = data
    for token in nlp(t):
        token.data = d
        token.source = n
        token_list.append(token)
    bar.update(i + 1)
bar.finish()

## Corpus Clustering 

In [19]:
def disjoin(_list, predicate, name = 'any feature'):
    a, b = [], []
    for item in _list:
        if predicate(item): a.append(item)
        else: b.append(item)

    print(name, len(a), len(b))
    return a, b

In [29]:
emojis, tokens = disjoin(token_list, lambda x : x.is_emoji(), 'emoji')
hashtags, tokens = disjoin(tokens, lambda x: x.is_hashtag(), 'hashtag') 
user_tags, tokens = disjoin(tokens, lambda x: x.is_user_tag(), 'user_tag') 
symbol, tokens = disjoin(tokens, lambda x: x.is_symbol, 'symbol')
space, tokens = disjoin(tokens, lambda x: x.space() or len(x.text) == 0, 'space')
urls, tokens = disjoin(tokens, lambda x: x.is_url(), 'url')


emoji 620 14667
hashtag 126 14541
user_tag 4 14537
symbol 1870 12667
space 2305 10362
url 34 10328


In [34]:
unknown, _ = disjoin(tokens, lambda x: x.unknown())

# for token in unknown:
#     print(token.text, end=' |')

unknown[0].text

any feature 93 10235


'️'

In [16]:
## numbers
## @
## symbols

In [35]:
natural, _ = disjoin(tokens, lambda x: x.natural_word())

any feature 9749 579


In [43]:
error = []
correct = []

bar = get_progressbar(len(natural), ' dictionary search ')
bar.start()
for i, token in enumerate(natural):
    obj = word_db.get(token.text)
    if obj is None or obj == 'None' or len(obj) == 0: continue
    if len(obj['rae']) == 1: error.append(token)
    else: correct.append((token, obj['rae']['articles']))
    bar.update(i + 1)
bar.finish()

print(len(error), len(correct))

574 6758


In [46]:
one_def, mult_def = disjoin(correct, lambda x: len(x[1]) == 1, 'one def')

one def 4698 2060


In [47]:
# get etymology
# get teclado error
# get neutral error 
# get english word

## Word Frequency

In [53]:
def sorted_by_dict(d):
    result = [(key, d[key] )for key in d]
    result.sort(key= lambda x: x[1], reverse= True)
    return result

def count_tokens (tokens, predicate = lambda x: True, name = ' count any feature '):
    bar = get_progressbar(len(tokens), name)

    result = {}
    bar.start()
    for i, t in enumerate(tokens):
        if not predicate(t): continue
        try: result[t.text.lower()] += 1
        except KeyError: 
            result[t.text.lower()] = 1
            
        bar.update(i + 1)
    bar.finish()

    return result, sorted_by_dict(result)

In [56]:
d, count = count_tokens(emojis)

print_best_word(count)

('‼', 52) ('👇', 29) ('🚨', 27) ('❤', 25) ('🐬', 20) ('👌', 19) ('🛑', 15) ('🐴', 15) ('👉', 14) ('📲', 13) ('🏻', 13) ('👈', 12) ('🎉', 11) ('🚌', 11) ('🙌', 9) ('😉', 8) ('🤣', 8) ('📱', 7) ('✅', 7) ('💯', 7) 

## Word Time Distribution 

In [51]:
def density_time(tokens, predicate = lambda x: True, name= ' count any feature '):
    bar = get_progressbar(len(tokens), name)

    result = {}
    bar.start()
    for i, t in enumerate(tokens):
        if not predicate(t): continue
        date = str(datetime.strptime(t.data.split(' ')[0], '%Y-%m-%d'))
        try: result[date][t.text.lower()] += 1
        except KeyError: 
            try: result[date][t.text.lower()] = 1
            except KeyError: 
                result[date] = {}
                result[date][t.text.lower()] = 1
            
        bar.update(i + 1)
    bar.finish()

    return result

def density_text_by_time(texts, name= ' count any feature '):
    bar = get_progressbar(len(texts), name)

    result = {}
    bar.start()
    for i, t in enumerate(texts):
        t, d, _ = t
        date = str(datetime.strptime(d.split(' ')[0], '%Y-%m-%d'))
        try: result[date] += 1
        except KeyError: 
            result[date] = 1
            
        bar.update(i + 1)
    bar.finish()

    return result

In [52]:
density_text = density_text_by_time(corpus)
density_tokens = density_time(tokens)

[######################################################] count any feature 100%
[######################################################] count any feature 100%
